# Investigation of Methods of Storing Data in `numpy` Arrays on Disk

In [1]:
# Remove h5 files if they exist
! rm -f *h5

In [72]:
from sys import getsizeof

import tables
import numpy as np
from copy import copy
from itertools import chain
from sklearn.linear_model import Perceptron
from sklearn.feature_extraction import (FeatureHasher,
                                        DictVectorizer)

from src import parse_non_nlp_features_string
from src.mongodb import connect_to_db
from src.experiments import (make_cursor,
                             get_data_point,
                             ExperimentalData)
from src.datasets import get_bin_ranges_helper

In [3]:
# Connect to reviews database
db = connect_to_db(host='localhost', port=37017)

In [4]:
# Get some data for "Dota_2" with the label "total_game_hours" and with
# the number of bins set to 3 and the bin factor set to 6.0
game = 'Dota_2'
label = 'total_game_hours'
nbins = 3
bin_factor = 6.0
#bin_ranges = get_bin_ranges_helper(db, [game], label, nbins, bin_factor)
bin_ranges = [(0.0, 382.2), (382.3, 2675.5), (2675.6, 16435.0)]
print("bin_ranges = {}".format(bin_ranges))

bin_ranges = [(0.0, 382.2), (382.3, 2675.5), (2675.6, 16435.0)]


In [5]:
data = ExperimentalData(db=db, prediction_label=label, games=[game],
                        folds=1, fold_size=30, grid_search_folds=0,
                        grid_search_fold_size=0, bin_ranges=bin_ranges)

In [6]:
# Now we have a set of IDs to work with, which point to samples in the
# dataset
data.training_set

[array(['5690a60fe76db81bef5c4613', '5690a60fe76db81bef5c30ed',
        '5690a60fe76db81bef5c322d', '5690a60fe76db81bef5c49ce',
        '5690a60fe76db81bef5c28ce', '5690a60fe76db81bef5c44b0',
        '5690a60fe76db81bef5c4987', '5690a60fe76db81bef5c496c',
        '5690a60fe76db81bef5c3e3f', '5690a60fe76db81bef5c4b13',
        '5690a60fe76db81bef5c403e', '5690a60fe76db81bef5c3cec',
        '5690a60fe76db81bef5c2e06', '5690a60fe76db81bef5c4464',
        '5690a60fe76db81bef5c3fd2', '5690a60fe76db81bef5c3510',
        '5690a60fe76db81bef5c36bb', '5690a60fe76db81bef5c2f74',
        '5690a60fe76db81bef5c2f51', '5690a60fe76db81bef5c4479',
        '5690a60fe76db81bef5c4ad5', '5690a60fe76db81bef5c47ff',
        '5690a60fe76db81bef5c3f2a', '5690a60fe76db81bef5c3872',
        '5690a60fe76db81bef5c2a3f', '5690a60fe76db81bef5c4465',
        '5690a60fe76db81bef5c2750', '5690a60fe76db81bef5c46c6',
        '5690a60fe76db81bef5c2aba', '5690a60fe76db81bef5c3973'], 
       dtype='<U24')]

In [7]:
data_ids = list(data.training_set[0])
data_ids[:10]

['5690a60fe76db81bef5c4613',
 '5690a60fe76db81bef5c30ed',
 '5690a60fe76db81bef5c322d',
 '5690a60fe76db81bef5c49ce',
 '5690a60fe76db81bef5c28ce',
 '5690a60fe76db81bef5c44b0',
 '5690a60fe76db81bef5c4987',
 '5690a60fe76db81bef5c496c',
 '5690a60fe76db81bef5c3e3f',
 '5690a60fe76db81bef5c4b13']

In [8]:
# For our features, we will use all of the NLP features + the review/reviewer
# attributes that are not directly related to the label ("total_game_hours")
non_nlp_feature_set_labels = parse_non_nlp_features_string("all", label)
print(non_nlp_feature_set_labels)

{'num_achievements_percentage', 'num_found_unhelpful', 'num_comments', 'num_workshop_items', 'num_screenshots', 'num_groups', 'num_voted_helpfulness', 'num_found_helpful', 'num_reviews', 'num_games_owned', 'found_helpful_percentage', 'friend_player_level', 'num_badges', 'num_found_funny', 'num_friends', 'num_achievements_possible', 'num_guides', 'num_achievements_attained'}


In [9]:
# Now we'll get the actual labels and the corresponding features for each
# sample
y = []
X = []
for doc in make_cursor(db, projection={'_id': 0}, id_strings=data_ids):
    sample = get_data_point(doc, prediction_label=label,
                            non_nlp_features=non_nlp_feature_set_labels,
                            bin_ranges=bin_ranges)
    if not sample: continue
    y.append(sample.get('y'))
    X.append(sample.get('x'))
y = np.array(y)
X = np.array(X)

In [10]:
# Example labels
y[:10]

array([2, 3, 2, 1, 2, 1, 1, 2, 1, 2])

In [11]:
# Example features (sparse format)
list(X[0].items())[:10]

[('community coupled', 1),
 ('any child', 1),
 ('show that', 1),
 ('long catch', 1),
 ("n't respect", 1),
 ('doing sport', 1),
 ("n't needed", 1),
 ('served', 1),
 ('barely listen', 1),
 ('automated', 1)]

In [68]:
# Types of values found in data
# Perhaps they can be represented instead with a less precise data-type than
# the current (float64)
set(chain(*[[val for _, val in x.items()] for x in X]))

{0.007722007722007722,
 0.031746031746031744,
 0.03571428571428571,
 0.09615384615384616,
 0.16216216216216217,
 0.16666666666666666,
 0.34782608695652173,
 0.5347490347490348,
 0.5454545454545454,
 0.625,
 0.6521739130434783,
 0.6666666666666666,
 0.6931818181818182,
 0.7142857142857143,
 0.7176470588235294,
 0.718562874251497,
 0.75,
 0.7659574468085106,
 0.7904191616766467,
 0.8,
 0.9003164556962026,
 0.9105398457583548,
 0.9655172413793104,
 0.9820359281437125,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 23,
 24,
 27,
 29,
 30,
 31,
 33,
 34,
 36,
 39,
 40,
 41,
 43,
 44,
 46,
 47,
 48,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 61,
 63,
 64,
 65,
 66,
 69,
 70,
 80,
 85,
 88,
 90,
 97,
 107,
 120,
 127,
 132,
 136,
 141,
 145,
 147,
 156,
 164,
 167,
 169,
 174,
 182,
 183,
 203,
 217,
 277,
 288,
 295,
 333,
 360,
 518,
 569,
 632,
 1771,
 1945}

In [12]:
# Now we'll make a vectorizer object (actually, two vectors, one with `DictVectorizer`
# and the other with `FeatureHasher`) and fit with `X`
dict_vec = DictVectorizer(sparse=True)
feature_hasher_vec = FeatureHasher(n_features=2**18, non_negative=True)

In [13]:
dict_vec.fit(X)
feature_hasher_vec.fit(X)

FeatureHasher(dtype=<class 'numpy.float64'>, input_type='dict',
       n_features=262144, non_negative=True)

In [14]:
feature_hasher_vec

FeatureHasher(dtype=<class 'numpy.float64'>, input_type='dict',
       n_features=262144, non_negative=True)

In [15]:
# Now we must transform `X` with the vectorizers to get the sparse scipy arrays
X_dict_vectorized = dict_vec.transform(X)
X_feature_hasher_vectorized = feature_hasher_vec.transform(X)

In [16]:
X_dict_vectorized

<30x150447 sparse matrix of type '<class 'numpy.float64'>'
	with 4223615 stored elements in Compressed Sparse Row format>

In [17]:
type(X_dict_vectorized)

scipy.sparse.csr.csr_matrix

In [18]:
type(X_feature_hasher_vectorized)

scipy.sparse.csr.csr_matrix

In [19]:
X_dict_vectorized.indptr

array([      0,  140430,  280869,  421947,  562705,  706627,  847056,
        987834, 1128189, 1268661, 1409102, 1549611, 1689989, 1830817,
       1971240, 2111773, 2252133, 2392525, 2532868, 2673419, 2814388,
       2954836, 3096452, 3237277, 3378794, 3521468, 3662042, 3802450,
       3942804, 4083194, 4223615], dtype=int32)

In [20]:
X_feature_hasher_vectorized.indptr

array([      0,  108600,  217212,  326169,  434956,  545560,  654160,
        762970,  871527,  980153, 1088762, 1197416, 1305980, 1414819,
       1523405, 1632054, 1740618, 1849192, 1957736, 2066414, 2175328,
       2283926, 2393221, 2502043, 2611281, 2721157, 2829832, 2938418,
       3046972, 3155548, 3264138], dtype=int32)

## Use `pytables` to Store `scipy` Arrays to Disk

- `scipy` sparse arrays unfortunately cannot be stored with `pytables`; however, they can be converted to dense arrays and then stored

In [21]:
# Open new empty HDF5 files
X_dict_vectorized_dense_file = tables.open_file("X_dict_vectorized_dense.h5", mode="w")
X_feature_hasher_dense_file = tables.open_file("X_feature_hasher_dense.h5", mode="w")

In [22]:
# Get the root groups
root_dict_vectorized = X_dict_vectorized_dense_file.root
root_feature_hasher_vectorized = X_feature_hasher_dense_file.root

In [23]:
# Save the dense arrays on the HDF5 files
X_dict_vectorized_dense_hdf_array = \
    X_dict_vectorized_dense_file.create_array(root_dict_vectorized,
                                              'X_dict_vectorized_dense',
                                              X_dict_vectorized.todense(),
                                              "X dict vectorized dense")
X_feature_hasher_dense_hdf_array = \
    X_feature_hasher_dense_file.create_array(root_feature_hasher_vectorized,
                                             'X_feature_hasher_dense',
                                             X_feature_hasher_vectorized.todense(),
                                             "X feature hasher vectorized dense")

In [24]:
! ls -lh X*_dense.h5

-rw-rw-r-- 1 mmulholland mmulholland 35M Mar  8 14:53 X_dict_vectorized_dense.h5
-rw-rw-r-- 1 mmulholland mmulholland 61M Mar  8 14:53 X_feature_hasher_dense.h5


- So, to store even a relatively small 30-sample dataset vectorized with `DictVectorizer` in a dense format, it can be 35 MB
- Suprisingly, storing the same dataset vectorized with `FeatureHasher`, which is supposed to be memory-efficient, requires even more memory to store the same data (61 MB) in a dense format
- Let's see what we can do with the array that's saved

In [25]:
X_dict_vectorized_dense_hdf_array

/X_dict_vectorized_dense (Array(30, 150447)) 'X dict vectorized dense'
  atom := Float64Atom(shape=(), dflt=0.0)
  maindim := 0
  flavor := 'numpy'
  byteorder := 'little'
  chunkshape := None

In [26]:
X_feature_hasher_dense_hdf_array

/X_feature_hasher_dense (Array(30, 262144)) 'X feature hasher vectorized dense'
  atom := Float64Atom(shape=(), dflt=0.0)
  maindim := 0
  flavor := 'numpy'
  byteorder := 'little'
  chunkshape := None

- Can a model be trained with `pytables` data?

In [27]:
perc1 = Perceptron()

In [28]:
perc1.fit(X_dict_vectorized_dense_hdf_array, y)

Perceptron(alpha=0.0001, class_weight=None, eta0=1.0, fit_intercept=True,
      n_iter=5, n_jobs=1, penalty=None, random_state=0, shuffle=True,
      verbose=0, warm_start=False)

In [29]:
perc1.predict(X_dict_vectorized_dense_hdf_array)

array([1, 2, 2, 1, 2, 2, 1, 2, 1, 1, 2, 2, 1, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 1, 2, 2])

In [30]:
perc2 = Perceptron()
perc2.fit(X_feature_hasher_dense_hdf_array, y)

Perceptron(alpha=0.0001, class_weight=None, eta0=1.0, fit_intercept=True,
      n_iter=5, n_jobs=1, penalty=None, random_state=0, shuffle=True,
      verbose=0, warm_start=False)

In [31]:
perc2.predict(X_feature_hasher_dense_hdf_array)

array([2, 2, 2, 1, 2, 2, 1, 2, 1, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 3, 2, 2, 1, 2, 2])

- Indeed, it seems that `pytables` data can be used as a drop-in replacement for non-sparse `numpy` arrays (or `todense`-converted sparse `scipy` arrays as generated via `DictVectorizer`/`FeatureHasher`

In [32]:
X_dict_vectorized_dense_file.close()
X_feature_hasher_dense_file.close()

- In order to use `pytables` in order to increase memory efficiency, a possible algorithm would be the following:
    - Extract and vectorize data
    - Use `todense` to make the arrays dense
    - Save to data an `hdf5` file with `pytables`
    - Remove the original data so that it gets garbage-collected
    - Use `pytables` arrays in place of data wherever needed
    - Remove the `hdf5` files after complete

- Note that it is possible to create enlargeable arrays with `pytables`, so it's possible that an array file could be generated, saved, and closed, and then reopened and enlargened and stored again.

- Now that the `hdf5` files have been created, arrays saved to them, and then closed, let's try to read in the data again and use it

In [33]:
X_dict_vectorized_dense_file = tables.open_file("X_dict_vectorized_dense.h5")
X_feature_hasher_dense_file = tables.open_file("X_feature_hasher_dense.h5")

In [34]:
X_dict_vectorized_dense_hdf_array = \
    X_dict_vectorized_dense_file.root.X_dict_vectorized_dense

In [35]:
X_dict_vectorized_dense_hdf_array

/X_dict_vectorized_dense (Array(30, 150447)) 'X dict vectorized dense'
  atom := Float64Atom(shape=(), dflt=0.0)
  maindim := 0
  flavor := 'numpy'
  byteorder := 'little'
  chunkshape := None

In [36]:
X_feature_hasher_dense_hdf_array = \
    X_feature_hasher_dense_file.root.X_feature_hasher_dense

In [37]:
X_feature_hasher_dense_hdf_array

/X_feature_hasher_dense (Array(30, 262144)) 'X feature hasher vectorized dense'
  atom := Float64Atom(shape=(), dflt=0.0)
  maindim := 0
  flavor := 'numpy'
  byteorder := 'little'
  chunkshape := None

In [38]:
perc1.fit(X_dict_vectorized_dense_hdf_array, y)

Perceptron(alpha=0.0001, class_weight=None, eta0=1.0, fit_intercept=True,
      n_iter=5, n_jobs=1, penalty=None, random_state=0, shuffle=True,
      verbose=0, warm_start=False)

In [39]:
perc2.fit(X_feature_hasher_dense_hdf_array, y)

Perceptron(alpha=0.0001, class_weight=None, eta0=1.0, fit_intercept=True,
      n_iter=5, n_jobs=1, penalty=None, random_state=0, shuffle=True,
      verbose=0, warm_start=False)

In [40]:
X_dict_vectorized_dense_file.close()
X_feature_hasher_dense_file.close()

### Using Compressed `pytables`
- `pytables` also exposes an `CArray` type that compresses the data
- A number of compression algorithms are provided, including `zlib`, `blosc`, and `lzo`
- Furthermore, memory can be optimized using `HD5`'s ability to handle in-memory processing via the `H5FD_CORE` driver

In [92]:
# This time, we'll create one table to store both arrays and we'll use `blosc` for
# compression
X_compressed_file = tables.open_file("X_compressed.h5", mode="w")
filters = tables.Filters(complevel=5, complib='blosc')
X_dict_vectorized_CArray = \
    X_compressed_file.create_carray(X_compressed_file.root,
                                    'X_dict_vectorized_CArray',
                                    tables.Atom.from_dtype(X_dict_vectorized.dtype),
                                    shape=X_dict_vectorized.shape,
                                    filters=filters)
X_dict_vectorized_CArray[:] = X_dict_vectorized.todense()
X_feature_hasher_CArray = \
    X_compressed_file.create_carray(X_compressed_file.root,
                                    'X_feature_hasher_CArray',
                                    tables.Atom.from_dtype(X_feature_hasher_vectorized.dtype),
                                    shape=X_feature_hasher_vectorized.shape,
                                    filters=filters)
X_feature_hasher_CArray[:] = X_feature_hasher_vectorized.todense()

ValueError: row, column, and data array must all be the same length

In [91]:
X_compressed_file.close()

- Now the size of `X_compressed.h5`, which contains both arrays, is only 8.6 MB!

In [43]:
# Let's read in the data from the table with the `H5FD_CORE` driver and train
# a model with it
X_compressed_file = tables.open_file("X_compressed.h5", mode='r', driver='H5FD_CORE')

In [44]:
X_dict_vectorized_CArray = X_compressed_file.root.X_dict_vectorized_CArray
X_feature_hasher_CArray = X_compressed_file.root.X_dict_vectorized_CArray

In [45]:
perc1 = Perceptron()
perc2 = Perceptron()
perc1.fit(X_dict_vectorized_CArray, y)
perc1.fit(X_feature_hasher_CArray, y)

Perceptron(alpha=0.0001, class_weight=None, eta0=1.0, fit_intercept=True,
      n_iter=5, n_jobs=1, penalty=None, random_state=0, shuffle=True,
      verbose=0, warm_start=False)

In [46]:
X_compressed_file.close()

### Using Compressed AND Extendable `pytables`
- `pytables` provides a further type of array that is both compressed and extendable, i.e., `EArray`

In [47]:
# Let's split up our data into 3 parts
data_ids_parts = []
for i in range(0, len(data_ids), 10):
    data_ids_parts.append(data_ids[i:i + 10])
[len(data_) for data_ in data_ids_parts]

[10, 10, 10]

In [48]:
# We will build up the `FeatureHasher` vectorizer first and in such a way that all
# of the data does not need to be held on disk at one time
vec = FeatureHasher(n_features=2**18, non_negative=True)
for data_ids_part in data_ids_parts:
    X_ = []
    for doc in make_cursor(db, projection={'_id': 0}, id_strings=data_ids_part):
        sample = get_data_point(doc, prediction_label=label,
                                non_nlp_features=non_nlp_feature_set_labels,
                                bin_ranges=bin_ranges)
        if not sample: continue
        X_.append(sample.get('x', sample))
    vec.fit(X_)
del X_

In [49]:
vec

FeatureHasher(dtype=<class 'numpy.float64'>, input_type='dict',
       n_features=262144, non_negative=True)

In [50]:
# Let's find out what the expected dtype/shape is supposed to be
X_ = []
for doc in make_cursor(db, projection={'_id': 0}, id_strings=data_ids[:5]):
    sample = get_data_point(doc, prediction_label=label,
                            non_nlp_features=non_nlp_feature_set_labels,
                            bin_ranges=bin_ranges)
    if not sample: continue
    X_.append(sample.get('x'))

In [51]:
X_ = vec.transform(X_).todense()

In [52]:
# Now that the vectorizer is fully fit, we can now read in the data
# incrementally, transform it, and save it to disk in a table
X_compressed_extendable_file = tables.open_file("X_compressed_extendable.h5", mode="w")
filters = tables.Filters(complevel=5, complib='blosc')

In [53]:
# Create array
X_earray = (X_compressed_extendable_file
            .create_earray(X_compressed_extendable_file.root,
                           'X_earray',
                           tables.Atom.from_dtype(X_.dtype),
                           shape=(0, X_.shape[1]),
                           filters=filters,
                           expectedrows=len(data_ids)))

In [54]:
X_earray

/X_earray (EArray(0, 262144), shuffle, blosc(5)) ''
  atom := Float64Atom(shape=(), dflt=0.0)
  maindim := 0
  flavor := 'numpy'
  byteorder := 'little'
  chunkshape := (1, 16384)

In [55]:
for data_ids_part in data_ids_parts:
    X_part = []
    for doc in make_cursor(db, projection={'_id': 0}, id_strings=data_ids_part):
        sample = get_data_point(doc, prediction_label=label,
                                non_nlp_features=non_nlp_feature_set_labels,
                                bin_ranges=bin_ranges)
        if not sample: continue
        X_part.append(sample.get('x'))
    X_earray.append(vec.transform(X_part).todense())

In [56]:
# Append rows to `X_earray` one at a time
for doc in make_cursor(db, projection={'_id': 0}, id_strings=data_ids):
    sample = get_data_point(doc, prediction_label=label,
                            non_nlp_features=non_nlp_feature_set_labels,
                            bin_ranges=bin_ranges)
    if not sample: continue
    X_ = vec.transform([sample.get('x')]).todense()
    X_earray.append(X_)

In [57]:
X_earray

/X_earray (EArray(60, 262144), shuffle, blosc(5)) ''
  atom := Float64Atom(shape=(), dflt=0.0)
  maindim := 0
  flavor := 'numpy'
  byteorder := 'little'
  chunkshape := (1, 16384)

In [58]:
! ls -lh X_compressed_extendable.h5

-rw-rw-r-- 1 mmulholland mmulholland 16M Mar  8 14:54 X_compressed_extendable.h5


In [59]:
X_compressed_extendable_file.close()

In [60]:
X_compressed_extendable_file = tables.open_file("X_compressed_extendable.h5",
                                                mode='r',
                                                driver='H5FD_CORE')

In [61]:
# Another way of referring to a specific dataset within a file (i.e., a "node"
# under the "root" of the file-system)
X_earray = X_compressed_extendable_file.get_node('/', 'X_earray')

In [62]:
X_earray[0]

array([ 2.,  2.,  1., ...,  2.,  0.,  1.])

In [63]:
X_compressed_extendable_file.close()